In [10]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)

In [3]:

def to_date_format(df):
    df['DT_NOTIFIC'] = pd.to_datetime(
        df['DT_NOTIFIC'], dayfirst=True)
    df['DT_NASC'] = pd.to_datetime(
        df['DT_NASC'], dayfirst=True)
    return df

In [4]:
path = "./data/dados_violencia_mulheres_ses"

files = [f for f in os.listdir(path) if f.endswith('.csv')]

dfs = [to_date_format(pd.read_csv(os.path.join(path, file), sep=';'))
       for file in files]


main = pd.concat(dfs, ignore_index=True)

In [13]:
main['NU_IDADE_N'] = main['NU_IDADE_N'].fillna(0).astype(int)

In [14]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388441 entries, 0 to 388440
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   DT_NOTIFIC  388441 non-null  datetime64[ns]
 1   DT_NASC     382825 non-null  datetime64[ns]
 2   NU_IDADE_N  388441 non-null  int64         
 3   CS_SEXO     388441 non-null  object        
 4   CS_RACA     382695 non-null  object        
 5   ID_MN_RESI  387495 non-null  object        
 6   LOCAL_OCOR  378460 non-null  object        
 7   OUT_VEZES   375551 non-null  object        
 8   LES_AUTOP   374831 non-null  object        
 9   VIOL_FISIC  377456 non-null  object        
 10  VIOL_PSICO  372815 non-null  object        
 11  VIOL_SEXU   372320 non-null  object        
 12  NUM_ENVOLV  373440 non-null  object        
 13  AUTOR_SEXO  376925 non-null  object        
 14  ORIENT_SEX  310969 non-null  object        
 15  IDENT_GEN   310969 non-null  object        
dtypes:

In [21]:
main.head()

,DT_NOTIFIC,DT_NASC,NU_IDADE_N,CS_SEXO,CS_RACA,ID_MN_RESI,LOCAL_OCOR,OUT_VEZES,LES_AUTOP,VIOL_FISIC,VIOL_PSICO,VIOL_SEXU,NUM_ENVOLV,AUTOR_SEXO,ORIENT_SEX,IDENT_GEN
0,2014-12-06,1952-10-08,62,Feminino,Preta,Corinto,Residencia,Não,Sim,Não,Não,Não,Um,Feminino,Não se aplica,Não se aplica
1,2014-09-23,1987-07-21,27,Feminino,Parda,Formoso,Residencia,NaN,NaN,Sim,Não,Não,Um,Masculino,NaN,NaN
2,2014-06-29,1997-09-06,16,Feminino,Parda,Belo Horizonte,Via pública,Sim,Não,Sim,Sim,Não,Dois ou mais,Masculino,Ignorado,Ignorado
3,2014-03-31,1980-08-12,33,Feminino,Parda,Ipatinga,NaN,Ignorado,Sim,Sim,NaN,NaN,Dois ou mais,NaN,NaN,NaN
4,2014-03-24,1980-08-12,33,Feminino,NaN,Ipatinga,Residencia,Ignorado,Sim,Sim,NaN,NaN,Um,Feminino,NaN,NaN


In [37]:
main['ID_MN_RESI'].value_counts()

ID_MN_RESI
Belo Horizonte                    28158
Betim                              9614
Uberaba                            9031
Uberlândia                         7977
Viçosa                             6047
Governador Valadares               6043
Contagem                           5988
Juiz de Fora                       5725
Conselheiro Lafaiete               5194
Ipatinga                           4805
Poços de Caldas                    3919
Manhuaçu                           3900
Varginha                           3863
Lavras                             3826
Itabira                            3773
Formiga                            3655
Ribeirão das Neves                 3411
Montes Claros                      3380
Santa Luzia                        3277
Congonhas                          3253
Pouso Alegre                       2844
Várzea da Palma                    2817
Sete Lagoas                        2676
Teófilo Otoni                      2654
Frutal                       

In [40]:
df_uber = main[main['ID_MN_RESI'] == 'Uberlândia']

In [47]:
df_group = df_uber.groupby(['ID_MN_RESI', 'NU_IDADE_N', 'CS_RACA', 'LOCAL_OCOR'])[['VIOL_FISIC', 'VIOL_PSICO', 'VIOL_SEXU']].count().unstack().reset_index()

In [48]:
df_melt = pd.melt(
    df_group,
    id_vars=['ID_MN_RESI', 'NU_IDADE_N', 'CS_RACA'],
    var_name='Violencia_Local',
    value_name='Count'
)

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['ID_MN_RESI', 'NU_IDADE_N', 'CS_RACA']"